<a href="https://colab.research.google.com/github/raquelbaldorab/Projeto-da-aula-1/blob/main/templates/aistudio_gemini_prompt_chat_b64.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6IkNyaWUgdW1hIHRhYmVsYSBkZSAxMFgxMCwgZW0gcXVlIGNhZGEgY8OpbHVsYSB0ZW5oYSB1bWEgbGV0cmEgZSB1bSBuw7ptZXJvLiJ9LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOiIjIyBUYWJlbGEgMTB4MTAgY29tIExldHJhcyBlIE7Dum1lcm9zOlxuXG58ICAgfCAxIHwgMiB8IDMgfCA0IHwgNSB8IDYgfCA3IHwgOCB8IDkgfCAxMCB8XG58LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXxcbnwgKipBKiogfCBBMSB8IEEyIHwgQTMgfCBBNCB8IEE1IHwgQTYgfCBBNyB8IEE4IHwgQTkgfCBBMTAgfFxufCAqKkIqKiB8IEIxIHwgQjIgfCBCMyB8IEI0IHwgQjUgfCBCNiB8IEI3IHwgQjggfCBCOSB8IEIxMCB8XG58ICoqQyoqIHwgQzEgfCBDMiB8IEMzIHwgQzQgfCBDNSB8IEM2IHwgQzcgfCBDOCB8IEM5IHwgQzEwIHxcbnwgKipEKiogfCBEMSB8IEQyIHwgRDMgfCBENCB8IEQ1IHwgRDYgfCBENyB8IEQ4IHwgRDkgfCBEMTAgfFxufCAqKkUqKiB8IEUxIHwgRTIgfCBFMyB8IEU0IHwgRTUgfCBFNiB8IEU3IHwgRTggfCBFOSB8IEUxMCB8XG58ICoqRioqIHwgRjEgfCBGMiB8IEYzIHwgRjQgfCBGNSB8IEY2IHwgRjcgfCBGOCB8IEY5IHwgRjEwIHxcbnwgKipHKiogfCBHMSB8IEcyIHwgRzMgfCBHNCB8IEc1IHwgRzYgfCBHNyB8IEc4IHwgRzkgfCBHMTAgfFxufCAqKkgqKiB8IEgxIHwgSDIgfCBIMyB8IEg0IHwgSDUgfCBINiB8IEg3IHwgSDggfCBIOSB8IEgxMCB8XG58ICoqSSoqIHwgSTEgfCBJMiB8IEkzIHwgSTQgfCBJNSB8IEk2IHwgSTcgfCBJOCB8IEk5IHwgSTEwIHxcbnwgKipKKiogfCBKMSB8IEoyIHwgSjMgfCBKNCB8IEo1IHwgSjYgfCBKNyB8IEo4IHwgSjkgfCBKMTAgfCJ9LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6IkludmVydGEgY29sb2NhbmRvIGFzIGxldHJhcyBubyBsdWdhciBkYXMgY29sdW5hcyBlIG9zIG7Dum1lcm9zIG5vIGx1Z2FyIGRhcyBsaW5oYXMuIn0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6IiMjIFRhYmVsYSAxMHgxMCBJbnZlcnRpZGE6XG5cbnwgICB8IEEgfCBCIHwgQyB8IEQgfCBFIHwgRiB8IEcgfCBIIHwgSSB8IEogfFxufC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18XG58ICoqMSoqIHwgMUEgfCAxQiB8IDFDIHwgMUQgfCAxRSB8IDFGIHwgMUcgfCAxSCB8IDFJIHwgMUogfFxufCAqKjIqKiB8IDJBIHwgMkIgfCAyQyB8IDJEIHwgMkUgfCAyRiB8IDJHIHwgMkggfCAySSB8IDJKIHxcbnwgKiozKiogfCAzQSB8IDNCIHwgM0MgfCAzRCB8IDNFIHwgM0YgfCAzRyB8IDNIIHwgM0kgfCAzSiB8XG58ICoqNCoqIHwgNEEgfCA0QiB8IDRDIHwgNEQgfCA0RSB8IDRGIHwgNEcgfCA0SCB8IDRJIHwgNEogfCBcbnwgKio1KiogfCA1QSB8IDVCIHwgNUMgfCA1RCB8IDVFIHwgNUYgfCA1RyB8IDVIIHwgNUkgfCA1SiB8IFxufCAqKjYqKiB8IDZBIHwgNkIgfCA2QyB8IDZEIHwgNkUgfCA2RiB8IDZHIHwgNkggfCA2SSB8IDZKIHxcbnwgKio3KiogfCA3QSB8IDdCIHwgN0MgfCA3RCB8IDdFIHwgN0YgfCA3RyB8IDdIIHwgN0kgfCA3SiB8XG58ICoqOCoqIHwgOEEgfCA4QiB8IDhDIHwgOEQgfCA4RSB8IDhGIHwgOEcgfCA4SCB8IDhJIHwgOEogfFxufCAqKjkqKiB8IDlBIHwgOUIgfCA5QyB8IDlEIHwgOUUgfCA5RiB8IDlHIHwgOUggfCA5SSB8IDlKIHxcbnwgKioxMCoqIHwgMTBBIHwgMTBCIHwgMTBDIHwgMTBEIHwgMTBFIHwgMTBGIHwgMTBHIHwgMTBIIHwgMTBJIHwgMTBKIHwifSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOiJtYW50ZW5oYSBhcyBsZXRyYXMgbmFzIGNvbHVuYXMsIG1hcyB0cm9xdWUgb3MgbsO6bWVyb3MgcGVsb3MgcmVzcGVjdGl2b3MgbcO6bHRpcGxvcyBkZSA3LCBzZWd1aW5kbyBvIGV4ZW1wbG8gQTcuIn0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6IiMjIFRhYmVsYSAxMHgxMCBjb20gTcO6bHRpcGxvcyBkZSA3OlxuXG58ICAgfCBBIHwgQiB8IEMgfCBEIHwgRSB8IEYgfCBHIHwgSCB8IEkgfCBKIHxcbnwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfFxufCAqKjcqKiB8IEE3IHwgQjcgfCBDNyB8IEQ3IHwgRTcgfCBGNyB8IEc3IHwgSDcgfCBJNyB8IEo3IHxcbnwgKioxNCoqIHwgQTE0IHwgQjE0IHwgQzE0IHwgRDE0IHwgRTE0IHwgRjE0IHwgRzE0IHwgSDE0IHwgSTE0IHwgSjE0IHxcbnwgKioyMSoqIHwgQTIxIHwgQjIxIHwgQzIxIHwgRDIxIHwgRTIxIHwgRjIxIHwgRzIxIHwgSDIxIHwgSTIxIHwgSjIxIHxcbnwgKioyOCoqIHwgQTI4IHwgQjI4IHwgQzI4IHwgRDI4IHwgRTI4IHwgRjI4IHwgRzI4IHwgSDI4IHwgSTI4IHwgSjI4IHwgXG58ICoqMzUqKiB8IEEzNSB8IEIzNSB8IEMzNSB8IEQzNSB8IEUzNSB8IEYzNSB8IEczNSB8IEgzNSB8IEkzNSB8IEozNSB8IFxufCAqKjQyKiogfCBBNDIgfCBCNDIgfCBDNDIgfCBENDIgfCBFNDIgfCBGNDIgfCBHNDIgfCBINDIgfCBJNDIgfCBKNDIgfFxufCAqKjQ5KiogfCBBNDkgfCBCNDkgfCBDNDkgfCBENDkgfCBFNDkgfCBGNDkgfCBHNDkgfCBINDkgfCBJNDkgfCBKNDkgfFxufCAqKjU2KiogfCBBNTYgfCBCNTYgfCBDNTYgfCBENTYgfCBFNTYgfCBGNTYgfCBHNTYgfCBINTYgfCBJNTYgfCBKNTYgfFxufCAqKjYzKiogfCBBNjMgfCBCNjMgfCBDNjMgfCBENjMgfCBFNjMgfCBGNjMgfCBHNjMgfCBINjMgfCBJNjMgfCBKNjMgfFxufCAqKjcwKiogfCBBNzAgfCBCNzAgfCBDNzAgfCBENzAgfCBFNzAgfCBGNzAgfCBHNzAgfCBINzAgfCBJNzAgfCBKNzAgfCJ9LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6ImFkaWNpb25lIHVtYSBjb2x1bmEgw6AgZGlyZWl0YSBuYSB0YWJlbGEgZW0gcXVlIG8gbsO6bWVybyBkYXMgY8OpbHVsYXMgc2VqYSBlcXVpdmFsZW50ZSBhIHNvbWEgZG9zIG7Dum1lcm9zICBkYSBsaW5oYS4ifSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjoiIyMgVGFiZWxhIDEweDEwIGNvbSBNw7psdGlwbG9zIGRlIDcgZSBTb21hOlxuXG58ICAgfCBBIHwgQiB8IEMgfCBEIHwgRSB8IEYgfCBHIHwgSCB8IEkgfCBKIHwgU29tYSB8XG58LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18LS0tfC0tLXwtLS18XG58ICoqNyoqIHwgQTcgfCBCNyB8IEM3IHwgRDcgfCBFNyB8IEY3IHwgRzcgfCBINyB8IEk3IHwgSjcgfCA3MCB8XG58ICoqMTQqKiB8IEExNCB8IEIxNCB8IEMxNCB8IEQxNCB8IEUxNCB8IEYxNCB8IEcxNCB8IEgxNCB8IEkxNCB8IEoxNCB8IDE0MCB8XG58ICoqMjEqKiB8IEEyMSB8IEIyMSB8IEMyMSB8IEQyMSB8IEUyMSB8IEYyMSB8IEcyMSB8IEgyMSB8IEkyMSB8IEoyMSB8IDIxMCB8XG58ICoqMjgqKiB8IEEyOCB8IEIyOCB8IEMyOCB8IEQyOCB8IEUyOCB8IEYyOCB8IEcyOCB8IEgyOCB8IEkyOCB8IEoyOCB8IDI4MCB8IFxufCAqKjM1KiogfCBBMzUgfCBCMzUgfCBDMzUgfCBEMzUgfCBFMzUgfCBGMzUgfCBHMzUgfCBIMzUgfCBJMzUgfCBKMzUgfCAzNTAgfCBcbnwgKio0MioqIHwgQTQyIHwgQjQyIHwgQzQyIHwgRDQyIHwgRTQyIHwgRjQyIHwgRzQyIHwgSDQyIHwgSTQyIHwgSjQyIHwgNDIwIHxcbnwgKio0OSoqIHwgQTQ5IHwgQjQ5IHwgQzQ5IHwgRDQ5IHwgRTQ5IHwgRjQ5IHwgRzQ5IHwgSDQ5IHwgSTQ5IHwgSjQ5IHwgNDkwIHxcbnwgKio1NioqIHwgQTU2IHwgQjU2IHwgQzU2IHwgRDU2IHwgRTU2IHwgRjU2IHwgRzU2IHwgSDU2IHwgSTU2IHwgSjU2IHwgNTYwIHxcbnwgKio2MyoqIHwgQTYzIHwgQjYzIHwgQzYzIHwgRDYzIHwgRTYzIHwgRjYzIHwgRzYzIHwgSDYzIHwgSTYzIHwgSjYzIHwgNjMwIHxcbnwgKio3MCoqIHwgQTcwIHwgQjcwIHwgQzcwIHwgRDcwIHwgRTcwIHwgRjcwIHwgRzcwIHwgSDcwIHwgSTcwIHwgSjcwIHwgNzAwIHwifSx7InJvbGUiOiJ1c2VyIiwicGFydHMiOiJmYcOnYSBjb20gcXVlIGEgdGFiZWxhIGFsdGVyZSBvcyB2YWxvcmVzIGRhIGNvbHVuYSBKIHNlIG9zIHZhbG9yZXMgZGFzIGPDqWx1bGFzIGRlIEEgYXTDqSBJIGZvcmVtIGFsdGVyYWRvcyBlIHZpY2UtdmVyc2EuIn0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6IiMjIEFsdGVyYcOnw6NvIERpbsOibWljYSBuYSBUYWJlbGFcblxuSW5mZWxpem1lbnRlLCBjb21vIHVtIG1vZGVsbyBkZSBsaW5ndWFnZW0sIG7Do28gdGVuaG8gYSBjYXBhY2lkYWRlIGRlIGNyaWFyIHRhYmVsYXMgaW50ZXJhdGl2YXMgZW0gdGVtcG8gcmVhbCBvbmRlIGFzIGFsdGVyYcOnw7VlcyBlbSB1bWEgY29sdW5hIGFmZXRhbSBhdXRvbWF0aWNhbWVudGUgb3MgdmFsb3JlcyBkZSBvdXRyYS4gXG5cbk5vIGVudGFudG8sIHBvc3NvIGZvcm5lY2VyIGEgdm9jw6ogdW0gY8OzZGlnbyBlbSBQeXRob24gcXVlIGNyaWEgdW1hIHRhYmVsYSBzZW1lbGhhbnRlIGUgZGVtb25zdHJhIGEgbMOzZ2ljYSBwYXJhIGF0dWFsaXphciBvcyB2YWxvcmVzIGRhIGNvbHVuYSBKIGNvbSBiYXNlIG5hcyBhbHRlcmHDp8O1ZXMgbmFzIGNvbHVuYXMgZGUgQSBhIEksIGUgdmljZS12ZXJzYS5cblxuYGBgcHl0aG9uXG5pbXBvcnQgcGFuZGFzIGFzIHBkXG5cbiMgQ3JpYXIgYSB0YWJlbGEgaW5pY2lhbFxuZGF0YSA9IHsnQSc6IFs3KmkgZm9yIGkgaW4gcmFuZ2UoMSwgMTEpXSxcbiAgICAgICAgJ0InOiBbNyppIGZvciBpIGluIHJhbmdlKDEsIDExKV0sXG4gICAgICAgICdDJzogWzcqaSBmb3IgaSBpbiByYW5nZSgxLCAxMSldLFxuICAgICAgICAnRCc6IFs3KmkgZm9yIGkgaW4gcmFuZ2UoMSwgMTEpXSxcbiAgICAgICAgJ0UnOiBbNyppIGZvciBpIGluIHJhbmdlKDEsIDExKV0sXG4gICAgICAgICdGJzogWzcqaSBmb3IgaSBpbiByYW5nZSgxLCAxMSldLFxuICAgICAgICAnRyc6IFs3KmkgZm9yIGkgaW4gcmFuZ2UoMSwgMTEpXSxcbiAgICAgICAgJ0gnOiBbNyppIGZvciBpIGluIHJhbmdlKDEsIDExKV0sXG4gICAgICAgICdJJzogWzcqaSBmb3IgaSBpbiByYW5nZSgxLCAxMSldLFxuICAgICAgICAnSic6IFs3KmkgZm9yIGkgaW4gcmFuZ2UoMSwgMTEpXX1cbmRmID0gcGQuRGF0YUZyYW1lKGRhdGEpXG5kZlsnU29tYSddID0gZGYuc3VtKGF4aXM9MSlcblxuIyBGdW7Dp8OjbyBwYXJhIGF0dWFsaXphciBvcyB2YWxvcmVzXG5kZWYgdXBkYXRlX3ZhbHVlcyhyb3csIGNvbCwgbmV3X3ZhbHVlKTpcbiAgICBkZi5sb2Nbcm93LCBjb2xdID0gbmV3X3ZhbHVlXG4gICAgIyBBdHVhbGl6YXIgYSBjb2x1bmEgSiBjb20gYmFzZSBuYSBzb21hIGRhcyBvdXRyYXMgY29sdW5hc1xuICAgIGRmLmxvY1tyb3csICdKJ10gPSBkZi5sb2Nbcm93LCAnU29tYSddIC0gZGYubG9jW3JvdywgJ0EnOidJJ10uc3VtKClcbiAgICAjIEF0dWFsaXphciBhIHNvbWFcbiAgICBkZi5sb2Nbcm93LCAnU29tYSddID0gZGYubG9jW3JvdywgOl0uc3VtKClcblxuIyBFeGVtcGxvIGRlIHVzb1xudXBkYXRlX3ZhbHVlcygyLCAnQycsIDEwMCkgICMgQWx0ZXJhciBvIHZhbG9yIG5hIGxpbmhhIDMsIGNvbHVuYSBDIHBhcmEgMTAwXG5wcmludChkZilcbmBgYFxuXG5Fc3RlIGPDs2RpZ28gdXRpbGl6YSBhIGJpYmxpb3RlY2EgUGFuZGFzIHBhcmEgY3JpYXIgZSBtYW5pcHVsYXIgYSB0YWJlbGEuIEEgZnVuw6fDo28gYHVwZGF0ZV92YWx1ZXNgIHBlcm1pdGUgYWx0ZXJhciBvIHZhbG9yIGRlIHVtYSBjw6lsdWxhIGUgYXR1YWxpemEgYXV0b21hdGljYW1lbnRlIGEgY29sdW5hIEogZSBhIHNvbWEgZGEgbGluaGEuIFxuXG5MZW1icmUtc2UgZGUgcXVlIGVzdGUgY8OzZGlnbyDDqSBhcGVuYXMgdW0gZXhlbXBsbyBlIHByZWNpc2Egc2VyIGV4ZWN1dGFkbyBlbSB1bSBhbWJpZW50ZSBQeXRob24gY29tIGEgYmlibGlvdGVjYSBQYW5kYXMgaW5zdGFsYWRhLiJ9LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6ImNyaWUgdW0gZXhlbXBsbyBwYXIgc2VyIGV4ZWN1dGFkbyBlbSB1bSBhbWJpZW50ZSBFeGNlbCJ9LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOiIjIyBFeGVtcGxvIHBhcmEgRXhjZWw6XG5cbk5vIEV4Y2VsLCBuw6NvIHBvZGVtb3MgY3JpYXIgdW1hIGF0dWFsaXphw6fDo28gYXV0b23DoXRpY2EgZGUgdmFsb3JlcyBlbnRyZSBjw6lsdWxhcyBzZW0gbyB1c28gZGUgbWFjcm9zIG91IFZCQS4gIEVudHJldGFudG8sIHBvZGVtb3MgdXRpbGl6YXIgZsOzcm11bGFzIHBhcmEgYXRpbmdpciB1bSByZXN1bHRhZG8gc2ltaWxhci5cblxuKipTaWdhIG9zIHBhc3NvcyBhYmFpeG86KipcblxuMS4gKipDcmllIGEgdGFiZWxhOioqIFJlcHJvZHV6YSBhIHRhYmVsYSAxMHgxMCBjb20gYXMgbGV0cmFzIG5hcyBjb2x1bmFzIGUgb3MgbcO6bHRpcGxvcyBkZSA3IG5hcyBsaW5oYXMsIGluY2x1aW5kbyBhIGNvbHVuYSBcIlNvbWFcIi5cbjIuICoqQ2FsY3VsZSBhIHNvbWEgZGEgbGluaGE6KiogTmEgY8OpbHVsYSBcIkwyXCIsIGluc2lyYSBhIGbDs3JtdWxhIGA9U1VNKEIyOksyKWAgZSBhcnJhc3RlIHBhcmEgYmFpeG8gcGFyYSBhcGxpY2FyIGEgZsOzcm11bGEgYSB0b2RhcyBhcyBsaW5oYXMuIElzc28gaXLDoSBjYWxjdWxhciBhIHNvbWEgZG9zIHZhbG9yZXMgZGUgY2FkYSBsaW5oYSwgZXhjbHVpbmRvIGEgY29sdW5hIEouXG4zLiAqKkNhbGN1bGUgbyB2YWxvciBkYSBjb2x1bmEgSjoqKiBOYSBjw6lsdWxhIFwiSzJcIiwgaW5zaXJhIGEgZsOzcm11bGEgYD1MMi1TVU0oQjI6SjIpYCBlIGFycmFzdGUgcGFyYSBiYWl4by4gRXN0YSBmw7NybXVsYSBzdWJ0cmFpIGEgc29tYSBkYXMgY29sdW5hcyBkZSBCIGEgSiAoaW5jbHVpbmRvIG8gcHLDs3ByaW8gSikgZGEgc29tYSB0b3RhbCBkYSBsaW5oYSAoTDIpLCByZXN1bHRhbmRvIG5vIHZhbG9yIGNvcnJldG8gcGFyYSBKLlxuXG4qKkV4ZW1wbG8gZGUgYXR1YWxpemHDp8OjbzoqKlxuXG4xLiAqKkFsdGVyZSB1bSB2YWxvcjoqKiBEaWdhbW9zIHF1ZSB2b2PDqiBhbHRlcmUgbyB2YWxvciBkYSBjw6lsdWxhIEMzIHBhcmEgMTAwLlxuMi4gKipBdHVhbGl6YcOnw6NvIGF1dG9tw6F0aWNhOioqIEEgY8OpbHVsYSBMMyAoc29tYSBkYSBsaW5oYSkgZSBhIGPDqWx1bGEgSzMgKHZhbG9yIGRlIEopIHNlcsOjbyBhdHVhbGl6YWRhcyBhdXRvbWF0aWNhbWVudGUgY29tIGJhc2UgbmFzIG5vdmFzIGbDs3JtdWxhcy5cblxuKipPYnNlcnZhw6fDtWVzOioqXG5cbiogRXN0YSBzb2x1w6fDo28gbsOjbyBhdHVhbGl6YSBhcyBjw6lsdWxhcyBkYSBjb2x1bmEgSiBhdXRvbWF0aWNhbWVudGUgc2Ugdm9jw6ogYWx0ZXJhciBkaXJldGFtZW50ZSBvIHZhbG9yIGRlIHVtYSBjw6lsdWxhIG5hIGNvbHVuYSBKLlxuKiBWb2PDqiBwb2RlIG9jdWx0YXIgYSBjb2x1bmEgXCJTb21hXCIgKEwpIHNlIGRlc2VqYXIsIHBvaXMgZWxhIMOpIHVzYWRhIGFwZW5hcyBwYXJhIGPDoWxjdWxvcyBpbnRlcm5vcy5cblxuKipBbHRlcm5hdGl2YXMgY29tIFZCQToqKlxuXG5Db20gbyB1c28gZGUgbWFjcm9zIGUgVkJBLCDDqSBwb3Nzw612ZWwgY3JpYXIgdW1hIHNvbHXDp8OjbyBtYWlzIGRpbsOibWljYSBxdWUgYXR1YWxpemUgYXV0b21hdGljYW1lbnRlIGFzIGPDqWx1bGFzIGRlIEEgYSBJIHF1YW5kbyBhIGNvbHVuYSBKIGZvciBhbHRlcmFkYS4gIE5vIGVudGFudG8sIGlzc28gcmVxdWVyIGNvbmhlY2ltZW50byBkZSBwcm9ncmFtYcOnw6NvIGVtIFZCQS5cblxuKipFc3Blcm8gcXVlIGVzdGUgZXhlbXBsbyBzZWphIMO6dGlsIHBhcmEgdm9jw6ohKioifV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [ ]:
contents

[{'role': 'user', 'parts': [{'text': 'hello'}]},
 {'role': 'model', 'parts': [{'text': 'Hello! How can I assist you today?'}]}]

In [ ]:
generation_config

{}

In [ ]:
safety_settings

{}

In [ ]:
user_input

'How does electricity work?'

### Call the API

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [ ]:
display(Markdown(response.text))

Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.

**Basic Principles of Electricity:**

1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.

2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.

3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).

4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).

**How Electricity Works:**

1. **Generation of Electricity:** Electricity can be generated in various ways, including:
   - **Mechanical Energy:** Generators convert mechanical energy into electrical energy by spinning a conductor in a magnetic field. This is the principle behind power plants that use turbines driven by steam, water, or wind.
   - **Chemical Energy:** Batteries generate electricity through chemical reactions.
   - **Solar Energy:** Solar cells convert sunlight directly into electricity using the photovoltaic effect.

2. **Transmission of Electricity:** Electricity is transmitted over long distances through power lines. These lines are made of conductive materials, such as copper or aluminum, which allow electrons to flow easily.

3. **Distribution of Electricity:** Once electricity reaches populated areas, it is distributed to homes, businesses, and other consumers through a network of local power lines and transformers.

4. **Utilization of Electricity:** Electricity is used to power various devices and appliances by converting electrical energy into other forms of energy, such as light, heat, or motion.

The flow of electricity in a circuit is driven by the difference in electric potential between two points. When a complete circuit is formed, electrons can flow from a region of higher potential to a region of lower potential, creating an electric current.

Electricity is a versatile form of energy that has revolutionized modern society. It is used in a wide range of applications, from lighting and heating to powering computers and transportation systems.

In [ ]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

In [ ]:
response.candidates

[index: 0
content {
  parts {
    text: "Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.\n\n**Basic Principles of Electricity:**\n\n1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.\n\n2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.\n\n3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).\n\n4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).\n\n**How Electricity Works:**\n\n1. **Generation of Electricity:** Electr